# Trial 1

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import random

2022-10-30 16:10:33.469264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.random.set_seed(21)
random.seed(21)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [4]:
df = pd.read_csv('../src/Fex_b.csv')

In [5]:
def read_dataframe(dataframe):
    dfc = dataframe.copy(True)
    dfc.drop('ImagesName', inplace=True, axis=1)
    return dfc

In [6]:
dfc = read_dataframe(df)

In [7]:
dfc

,CDR,VCDR,RDR,I,S,N,T,Labels
0,0.602564,0.470000,0.071795,236,218,130,133,0
1,0.493056,0.423459,0.060185,197,241,127,173,0
2,0.513298,0.465060,0.154255,172,154,133,116,0
3,0.450000,0.432911,0.121053,161,165,99,169,0
4,0.447570,0.418660,0.132992,173,187,120,170,0
...,...,...,...,...,...,...,...,...
163,0.457921,0.456790,0.195545,160,141,137,141,1
164,0.501401,0.485095,0.193277,129,138,118,132,1
165,0.467262,0.480122,0.148810,131,121,104,134,1
166,0.365439,0.398148,0.195467,130,129,165,105,1


In [8]:
dfc[dfc['RDR'] != 2]['RDR'].mean()

0.11506752951139768

In [9]:
dfc.loc[dfc["RDR"] == 2.0, "RDR"] = dfc[dfc['RDR'] != 2]['RDR'].mean()

In [10]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# 5 -> 79%
model = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu'), # 4,5,7,1
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(14, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=50random.seed(3), verbose=0)

2022-10-30 16:10:38.770251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
predictions = model.predict(X_test_scaled)
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

2/2 [==============================] - 0s 2ms/step


In [14]:
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.44
Precision: 0.38
Recall: 0.57


## SVM

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [16]:
df = pd.read_csv('../src/Fex_b.csv')

In [17]:
def read_dataframe(dataframe):
    dfc = dataframe.copy(True)
    dfc.drop('ImagesName', inplace=True, axis=1)
    return dfc

In [18]:
dfc = read_dataframe(df)

In [19]:
dfc.loc[dfc["RDR"] == 2.0, "RDR"] = dfc[dfc['RDR'] != 2]['RDR'].mean()

In [20]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [21]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
clf = svm.SVC()
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

In [23]:
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
print(f'Precision: {precision_score(y_test, y_pred):.2f}')
print(f'Recall: {recall_score(y_test, y_pred):.2f}')

Accuracy: 0.68
Precision: 0.58
Recall: 0.79
